In [1]:
from vanna.openai import OpenAI_Chat
from vanna.chromadb import ChromaDB_VectorStore
import os
from dotenv import load_dotenv
load_dotenv()

class MyVanna(ChromaDB_VectorStore, OpenAI_Chat):
    def __init__(self, config=None):
        ChromaDB_VectorStore.__init__(self, config=config)
        OpenAI_Chat.__init__(self, config=config)

vn = MyVanna(config={'api_key':os.getenv('OPENAI_API_KEY'), 'model': 'gpt-4o-mini'})
# vn.remove_training_data()

vn.connect_to_postgres(host=os.getenv('POSTGRES_HOST'), dbname=os.getenv('POSTGRES_DB'), user=os.getenv('POSTGRES_USER'), password=os.getenv('POSTGRES_PASSWORD'), port=os.getenv('POSTGRES_PORT'))

In [2]:

# # Récupérer les métadonnées du schéma
df_information_schema = vn.run_sql("SELECT * FROM INFORMATION_SCHEMA.COLUMNS")

df_information_schema # Afficher la réponse brute

,table_catalog,table_schema,table_name,column_name,ordinal_position,column_default,is_nullable,data_type,character_maximum_length,character_octet_length,...,is_identity,identity_generation,identity_start,identity_increment,identity_maximum,identity_minimum,identity_cycle,is_generated,generation_expression,is_updatable
0,GMAO,pg_catalog,pg_type,typsubscript,13,None,NO,regproc,NaN,NaN,...,NO,None,None,None,None,None,NO,NEVER,None,YES
1,GMAO,pg_catalog,pg_class,relpersistence,16,None,NO,"""char""",NaN,NaN,...,NO,None,None,None,None,None,NO,NEVER,None,YES
2,GMAO,information_schema,tables,table_schema,2,None,YES,name,NaN,NaN,...,NO,None,None,None,None,None,NO,NEVER,None,NO
3,GMAO,public,demandes,commentaires,10,None,YES,text,NaN,1.073742e+09,...,NO,None,None,None,None,None,NO,NEVER,None,YES
4,GMAO,pg_catalog,pg_stat_ssl,client_dn,6,None,YES,text,NaN,1.073742e+09,...,NO,None,None,None,None,None,NO,NEVER,None,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2137,GMAO,information_schema,routine_column_usage,specific_catalog,1,None,YES,name,NaN,NaN,...,NO,None,None,None,None,None,NO,NEVER,None,NO
2138,GMAO,pg_catalog,pg_stat_replication,application_name,4,None,YES,text,NaN,1.073742e+09,...,NO,None,None,None,None,None,NO,NEVER,None,NO
2139,GMAO,pg_catalog,pg_subscription,subskiplsn,3,None,NO,pg_lsn,NaN,NaN,...,NO,None,None,None,None,None,NO,NEVER,None,YES
2140,GMAO,pg_catalog,pg_statistic_ext,stxnamespace,4,None,NO,oid,NaN,NaN,...,NO,None,None,None,None,None,NO,NEVER,None,YES


In [3]:

# # Générer le plan d'entraînement
plan = vn.get_training_plan_generic(df_information_schema)
print(plan)

# Entraîner le modèle
vn.train(plan=plan)



Train on Information Schema: GMAO.pg_catalog pg_type
Train on Information Schema: GMAO.pg_catalog pg_class
Train on Information Schema: GMAO.pg_catalog pg_stat_ssl
Train on Information Schema: GMAO.pg_catalog pg_index
Train on Information Schema: GMAO.pg_catalog pg_attribute
Train on Information Schema: GMAO.pg_catalog pg_stat_progress_cluster
Train on Information Schema: GMAO.pg_catalog pg_shseclabel
Train on Information Schema: GMAO.pg_catalog pg_indexes
Train on Information Schema: GMAO.pg_catalog pg_database
Train on Information Schema: GMAO.pg_catalog pg_amproc
Train on Information Schema: GMAO.pg_catalog pg_stat_progress_vacuum
Train on Information Schema: GMAO.pg_catalog pg_stat_database
Train on Information Schema: GMAO.pg_catalog pg_default_acl
Train on Information Schema: GMAO.pg_catalog pg_stat_user_tables
Train on Information Schema: GMAO.pg_catalog pg_largeobject
Train on Information Schema: GMAO.pg_catalog pg_statio_sys_tables
Train on Information Schema: GMAO.pg_catalog 

Add of existing embedding ID: 9decbdef-e99e-5832-a511-a41486da9273-doc
Insert of existing embedding ID: 9decbdef-e99e-5832-a511-a41486da9273-doc
Add of existing embedding ID: 27429a4d-e304-5df6-a7ba-3cefc7c415ff-doc
Insert of existing embedding ID: 27429a4d-e304-5df6-a7ba-3cefc7c415ff-doc
Add of existing embedding ID: 204869c8-8a80-5e1e-a8dd-80b40bea9fc6-doc
Insert of existing embedding ID: 204869c8-8a80-5e1e-a8dd-80b40bea9fc6-doc
Add of existing embedding ID: afa8c12c-78a7-51db-a39d-88963176d2fa-doc
Insert of existing embedding ID: afa8c12c-78a7-51db-a39d-88963176d2fa-doc
Add of existing embedding ID: 76d1a97e-7e0d-55ad-9e4b-79b582dd34cc-doc
Insert of existing embedding ID: 76d1a97e-7e0d-55ad-9e4b-79b582dd34cc-doc
Add of existing embedding ID: b70a307b-8ffd-54da-8c40-0fd7f051c40e-doc
Insert of existing embedding ID: b70a307b-8ffd-54da-8c40-0fd7f051c40e-doc
Add of existing embedding ID: affa5d35-fb7a-577f-a483-facbed929b33-doc
Insert of existing embedding ID: affa5d35-fb7a-577f-a483-fa

In [4]:
# Question simple - Liste des fournisseurs
vn.train(
    question="Liste-moi tous les fournisseurs",
    sql="SELECT nom_fournisseur, statut FROM fournisseurs ORDER BY nom_fournisseur;"
)

vn.train(
    question="Quels sont les fournisseurs actifs ?",
    sql="SELECT id_fournisseur, nom_fournisseur, contact1_email FROM fournisseurs WHERE statut = 'Actif';"
)

vn.train(
    question="Montre-moi les coordonnées du fournisseur X",
    sql="SELECT nom_fournisseur, adresse, contact1_nom, contact1_email, contact1_tel_mobile FROM fournisseurs WHERE nom_fournisseur ILIKE '%X%';"
)

vn.train(
    question="Combien de fournisseurs avons-nous ?",
    sql="SELECT COUNT(*) as nombre_fournisseurs FROM fournisseurs WHERE statut = 'Actif';"
)

# =====================================================
# QUESTIONS SUR L'INVENTAIRE/STOCK
# =====================================================

vn.train(
    question="Montre-moi tous les produits en stock",
    sql="SELECT reference, produits, quantite, emplacement, fournisseur FROM inventaire WHERE quantite > 0 ORDER BY produits;"
)

vn.train(
    question="Quels produits ont un stock faible ?",
    sql="SELECT reference, produits, quantite, stock_min, emplacement FROM inventaire WHERE quantite <= stock_min ORDER BY quantite;"
)

vn.train(
    question="Liste des produits par fournisseur",
    sql="SELECT fournisseur, COUNT(*) as nb_produits, SUM(quantite) as stock_total FROM inventaire GROUP BY fournisseur ORDER BY nb_produits DESC;"
)

vn.train(
    question="Quel est le stock total par catégorie ?",
    sql="SELECT categorie, COUNT(*) as nb_produits, SUM(quantite) as quantite_totale FROM inventaire GROUP BY categorie ORDER BY quantite_totale DESC;"
)

vn.train(
    question="Recherche un produit par référence",
    sql="SELECT * FROM inventaire WHERE reference = 'REF123';"
)

vn.train(
    question="Produits stockés dans l'emplacement X",
    sql="SELECT reference, produits, quantite, prix_unitaire FROM inventaire WHERE emplacement ILIKE '%X%';"
)

vn.train(
    question="Valeur totale du stock",
    sql="SELECT SUM(quantite * prix_unitaire) as valeur_totale_stock FROM inventaire;"
)

# =====================================================
# QUESTIONS SUR LA HIÉRARCHIE (SITES/LIEUX/EMPLACEMENTS)
# =====================================================

vn.train(
    question="Liste tous les sites",
    sql="SELECT code_site, nom_site, ville, responsable FROM sites WHERE statut = 'Actif';"
)

vn.train(
    question="Quels sont les lieux du site X ?",
    sql="SELECT l.code_lieu, l.nom_lieu, l.type_lieu, s.nom_site FROM lieux l JOIN sites s ON l.site_id = s.id WHERE s.nom_site ILIKE '%X%';"
)

vn.train(
    question="Montre-moi les emplacements du lieu Y",
    sql="SELECT e.code_emplacement, e.nom_emplacement, e.type_emplacement, e.nb_produits FROM emplacements e JOIN lieux l ON e.lieu_id = l.id WHERE l.nom_lieu ILIKE '%Y%';"
)

vn.train(
    question="Hiérarchie complète des emplacements",
    sql="SELECT s.nom_site, l.nom_lieu, e.nom_emplacement, e.nb_produits FROM sites s JOIN lieux l ON s.id = l.site_id JOIN emplacements e ON l.id = e.lieu_id ORDER BY s.nom_site, l.nom_lieu, e.nom_emplacement;"
)

# =====================================================
# QUESTIONS SUR LES DEMANDES
# =====================================================

vn.train(
    question="Liste des demandes en attente",
    sql="SELECT id_demande, date_demande, demandeur, statut FROM demandes WHERE statut = 'En attente' ORDER BY date_demande;"
)

vn.train(
    question="Demandes par demandeur",
    sql="SELECT demandeur, COUNT(*) as nb_demandes, MAX(date_demande) as derniere_demande FROM demandes GROUP BY demandeur ORDER BY nb_demandes DESC;"
)

vn.train(
    question="Demandes traitées ce mois",
    sql="SELECT COUNT(*) as demandes_traitees FROM demandes WHERE statut = 'Traitée' AND DATE_TRUNC('month', date_traitement) = DATE_TRUNC('month', CURRENT_DATE);"
)

# =====================================================
# QUESTIONS SUR L'HISTORIQUE
# =====================================================

vn.train(
    question="Historique des mouvements du produit X",
    sql="SELECT date_mouvement, nature, quantite_mouvement, quantite_avant, quantite_apres FROM historique WHERE produit ILIKE '%X%' ORDER BY date_mouvement DESC;"
)

vn.train(
    question="Mouvements d'entrée cette semaine",
    sql="SELECT reference, produit, quantite_mouvement, date_mouvement FROM historique WHERE nature = 'Entrée' AND date_mouvement >= DATE_TRUNC('week', CURRENT_DATE);"
)

vn.train(
    question="Mouvements de sortie par jour",
    sql="SELECT DATE(date_mouvement) as jour, COUNT(*) as nb_sorties, SUM(quantite_mouvement) as quantite_totale FROM historique WHERE nature = 'Sortie' GROUP BY DATE(date_mouvement) ORDER BY jour DESC;"
)

# =====================================================
# QUESTIONS SUR LES TABLES D'ATELIER
# =====================================================

vn.train(
    question="Liste des tables d'atelier",
    sql="SELECT id_table, nom_table, type_atelier, responsable FROM tables_atelier WHERE statut = 'Actif';"
)

vn.train(
    question="Tables d'atelier par responsable",
    sql="SELECT responsable, COUNT(*) as nb_tables FROM tables_atelier WHERE statut = 'Actif' GROUP BY responsable;"
)

# =====================================================
# QUESTIONS SUR LES LISTES D'INVENTAIRE
# =====================================================

vn.train(
    question="Listes d'inventaire en cours",
    sql="SELECT id_liste, nom_liste, statut, nb_produits, date_creation FROM listes_inventaire WHERE statut IN ('En préparation', 'En cours');"
)

vn.train(
    question="Produits dans la liste d'inventaire X",
    sql="SELECT reference_produit, nom_produit, quantite_theorique, quantite_comptee, emplacement FROM produits_listes_inventaire WHERE id_liste = 'X';"
)

# =====================================================
# QUESTIONS COMPLEXES AVEC JOINTURES
# =====================================================

vn.train(
    question="Produits et leurs fournisseurs avec contacts",
    sql="SELECT i.reference, i.produits, i.quantite, f.nom_fournisseur, f.contact1_email FROM inventaire i LEFT JOIN fournisseurs f ON i.fournisseur = f.nom_fournisseur WHERE i.quantite > 0;"
)

vn.train(
    question="Stock par site avec détails",
    sql="SELECT s.nom_site, l.nom_lieu, e.nom_emplacement, COUNT(i.id) as nb_produits, SUM(i.quantite) as stock_total FROM sites s JOIN lieux l ON s.id = l.site_id JOIN emplacements e ON l.id = e.lieu_id LEFT JOIN inventaire i ON CONCAT(s.nom_site, ' - ', l.nom_lieu, ' - ', e.nom_emplacement) = i.emplacement GROUP BY s.nom_site, l.nom_lieu, e.nom_emplacement ORDER BY stock_total DESC;"
)

vn.train(
    question="Analyse des ruptures de stock",
    sql="SELECT i.reference, i.produits, i.quantite, i.stock_min, f.nom_fournisseur, f.contact1_email FROM inventaire i LEFT JOIN fournisseurs f ON i.fournisseur = f.nom_fournisseur WHERE i.quantite <= i.stock_min ORDER BY (i.stock_min - i.quantite) DESC;"
)

# =====================================================
# QUESTIONS STATISTIQUES ET ANALYTIQUES
# =====================================================

vn.train(
    question="Top 10 des produits les plus stockés",
    sql="SELECT reference, produits, quantite, prix_unitaire, (quantite * prix_unitaire) as valeur FROM inventaire WHERE quantite > 0 ORDER BY quantite DESC LIMIT 10;"
)

vn.train(
    question="Répartition du stock par secteur",
    sql="SELECT secteur, COUNT(*) as nb_produits, SUM(quantite) as stock_total, AVG(prix_unitaire) as prix_moyen FROM inventaire WHERE secteur IS NOT NULL GROUP BY secteur ORDER BY stock_total DESC;"
)

vn.train(
    question="Évolution des entrées/sorties par mois",
    sql="SELECT DATE_TRUNC('month', date_mouvement) as mois, nature, COUNT(*) as nb_mouvements, SUM(quantite_mouvement) as quantite_totale FROM historique GROUP BY DATE_TRUNC('month', date_mouvement), nature ORDER BY mois DESC, nature;"
)

vn.train(
    question="Taux d'occupation des emplacements",
    sql="SELECT nom_emplacement, nb_produits, capacite_max, taux_occupation, (nb_produits::float / capacite_max * 100) as pourcentage_reel FROM emplacements WHERE capacite_max > 0 ORDER BY taux_occupation DESC;"
)

# =====================================================
# QUESTIONS DE RECHERCHE ET FILTRES
# =====================================================

vn.train(
    question="Recherche produit contenant 'vis'",
    sql="SELECT reference, produits, quantite, emplacement FROM inventaire WHERE produits ILIKE '%vis%' ORDER BY produits;"
)

vn.train(
    question="Produits ajoutés cette semaine",
    sql="SELECT reference, produits, quantite, date_entree FROM inventaire WHERE date_entree >= DATE_TRUNC('week', CURRENT_DATE) ORDER BY date_entree DESC;"
)

vn.train(
    question="Fournisseurs sans produits en stock",
    sql="SELECT f.nom_fournisseur, f.contact1_email FROM fournisseurs f LEFT JOIN inventaire i ON f.nom_fournisseur = i.fournisseur WHERE i.fournisseur IS NULL AND f.statut = 'Actif';"
)


Add of existing embedding ID: 6ad00744-f6f1-5fda-9243-b2de39cd6989-sql
Add of existing embedding ID: 25e202bb-2fcc-55f2-8a59-f4b872d71187-sql
Add of existing embedding ID: 352c3c4f-4364-57e0-a468-43d3b72c246b-sql
Add of existing embedding ID: 8b9021c1-0aa4-5cf1-8b0a-e0bcedc10a21-sql
Add of existing embedding ID: 6a580a8e-dc98-5e55-bcdf-e43c130baa0e-sql
Add of existing embedding ID: a2d56652-10b8-57b4-afdd-86aae9060c55-sql
Add of existing embedding ID: 7f41cbc3-77d9-5b7c-b4c0-e21509702257-sql
Add of existing embedding ID: a3a8a2c5-a0ae-5652-af1a-6459b89bc74d-sql
Add of existing embedding ID: f8b8dc1a-6cb1-5032-88c1-d84c16d370af-sql
Insert of existing embedding ID: 6ad00744-f6f1-5fda-9243-b2de39cd6989-sql
Add of existing embedding ID: 6ad00744-f6f1-5fda-9243-b2de39cd6989-sql
Insert of existing embedding ID: 25e202bb-2fcc-55f2-8a59-f4b872d71187-sql
Add of existing embedding ID: 25e202bb-2fcc-55f2-8a59-f4b872d71187-sql
Insert of existing embedding ID: 73971186-55fb-5ea7-bba4-d02fda0fd684-s

'cad29f19-371f-56ce-bfdd-7268e9f344b0-sql'

In [5]:
# =====================================================
# ENTRAÎNEMENT AVEC PAIRES QUESTION-SQL PERSONNALISÉES
# =====================================================

# Questions sur les fournisseurs
vn.train(
    question="Liste-moi tous les fournisseurs",
    sql="SELECT nom_fournisseur, statut FROM fournisseurs ORDER BY nom_fournisseur;"
)

vn.train(
    question="Quels sont les fournisseurs actifs ?",
    sql="SELECT id_fournisseur, nom_fournisseur, contact1_email FROM fournisseurs WHERE statut = 'Actif';"
)

vn.train(
    question="Combien de fournisseurs avons-nous ?",
    sql="SELECT COUNT(*) as nombre_fournisseurs FROM fournisseurs WHERE statut = 'Actif';"
)

# Questions sur l'inventaire
vn.train(
    question="Montre-moi tous les produits en stock",
    sql="SELECT reference, produits, quantite, emplacement, fournisseur FROM inventaire WHERE quantite > 0 ORDER BY produits;"
)

vn.train(
    question="Quels produits ont un stock faible ?",
    sql="SELECT reference, produits, quantite, stock_min, emplacement FROM inventaire WHERE quantite <= stock_min ORDER BY quantite;"
)

vn.train(
    question="Valeur totale du stock",
    sql="SELECT SUM(quantite * prix_unitaire) as valeur_totale_stock FROM inventaire;"
)

# Questions sur la hiérarchie
vn.train(
    question="Liste tous les sites",
    sql="SELECT code_site, nom_site, ville, responsable FROM sites WHERE statut = 'Actif';"
)

vn.train(
    question="Hiérarchie complète des emplacements",
    sql="SELECT s.nom_site, l.nom_lieu, e.nom_emplacement, e.nb_produits FROM sites s JOIN lieux l ON s.id = l.site_id JOIN emplacements e ON l.id = e.lieu_id ORDER BY s.nom_site, l.nom_lieu, e.nom_emplacement;"
)

# Questions complexes
vn.train(
    question="Produits et leurs fournisseurs avec contacts",
    sql="SELECT i.reference, i.produits, i.quantite, f.nom_fournisseur, f.contact1_email FROM inventaire i LEFT JOIN fournisseurs f ON i.fournisseur = f.nom_fournisseur WHERE i.quantite > 0;"
)

print("✅ Entraînement personnalisé terminé !")


Insert of existing embedding ID: 6ad00744-f6f1-5fda-9243-b2de39cd6989-sql
Add of existing embedding ID: 6ad00744-f6f1-5fda-9243-b2de39cd6989-sql
Insert of existing embedding ID: 25e202bb-2fcc-55f2-8a59-f4b872d71187-sql
Add of existing embedding ID: 25e202bb-2fcc-55f2-8a59-f4b872d71187-sql
Insert of existing embedding ID: 352c3c4f-4364-57e0-a468-43d3b72c246b-sql
Add of existing embedding ID: 352c3c4f-4364-57e0-a468-43d3b72c246b-sql
Insert of existing embedding ID: 8b9021c1-0aa4-5cf1-8b0a-e0bcedc10a21-sql
Add of existing embedding ID: 8b9021c1-0aa4-5cf1-8b0a-e0bcedc10a21-sql
Insert of existing embedding ID: 6a580a8e-dc98-5e55-bcdf-e43c130baa0e-sql
Add of existing embedding ID: 6a580a8e-dc98-5e55-bcdf-e43c130baa0e-sql
Insert of existing embedding ID: a2d56652-10b8-57b4-afdd-86aae9060c55-sql
Add of existing embedding ID: a2d56652-10b8-57b4-afdd-86aae9060c55-sql
Insert of existing embedding ID: 7f41cbc3-77d9-5b7c-b4c0-e21509702257-sql
Add of existing embedding ID: 7f41cbc3-77d9-5b7c-b4c0-e2

✅ Entraînement personnalisé terminé !


In [6]:
training_data = vn.get_training_data()
training_data



,id,question,content,training_data_type
0,6ad00744-f6f1-5fda-9243-b2de39cd6989-sql,Liste-moi tous les fournisseurs,"SELECT nom_fournisseur, statut FROM fournisseu...",sql
1,25e202bb-2fcc-55f2-8a59-f4b872d71187-sql,Quels sont les fournisseurs actifs ?,"SELECT id_fournisseur, nom_fournisseur, contac...",sql
2,73971186-55fb-5ea7-bba4-d02fda0fd684-sql,Montre-moi les coordonnées du fournisseur X,"SELECT nom_fournisseur, adresse, contact1_nom,...",sql
3,352c3c4f-4364-57e0-a468-43d3b72c246b-sql,Combien de fournisseurs avons-nous ?,SELECT COUNT(*) as nombre_fournisseurs FROM fo...,sql
4,8b9021c1-0aa4-5cf1-8b0a-e0bcedc10a21-sql,Montre-moi tous les produits en stock,"SELECT reference, produits, quantite, emplacem...",sql
...,...,...,...,...
213,f72f8fe2-e465-5834-ac0b-a893ce2964cf-doc,None,The following columns are in the historique ta...,documentation
214,a464fc8c-b39c-515d-848a-74d92fc5a79e-doc,None,The following columns are in the lieux table i...,documentation
215,862cfd88-e7b6-5684-aff9-5addad1ea280-doc,None,The following columns are in the produits_list...,documentation
216,7b83f2b2-06a5-5453-8f02-0a626fe4f58b-doc,None,The following columns are in the inventaire ta...,documentation


In [11]:
from vanna.flask import VannaFlaskApp
VannaFlaskApp(vn, allow_llm_to_see_data=True).run()

Your app is running at:
http://localhost:8084
 * Serving Flask app 'vanna.flask'
 * Debug mode: on


Address already in use
Port 8084 is in use by another program. Either identify and stop that program, or start the server with a different port.


SystemExit: 1